In [1]:
from astropy.coordinates import SkyCoord
import astropy.units as u
from IPython.display import display, HTML
import pandas as pd
import numpy as np

# import observations of selected objects outputted midway from sorcha
obs_df = pd.read_csv('./midobs1000tnos.csv')
obs_df = obs_df.sort_values(by=['ssObjectId'])
obs_df = obs_df.reset_index(drop=True)
obs_df['ssObjectId'] = obs_df['ssObjectId'].astype(str)


# convert to record array for sorcha compatability
nameLen = obs_df['ssObjectId'].str.len().max()
obsv = obs_df.to_records(
    index=False,
    column_dtypes=dict(ssObjectId=f'a{nameLen}', diaSourceId='u8', midPointTai='f8', ra='f8', decl='f8')
)

# split record array by object
i = np.argsort(obsv['ssObjectId'], kind='stable')
ssObjects, idx = np.unique(obsv['ssObjectId'][i], return_index=True)
objSplits = np.split(i, idx[1:])

tracklets = []
a=0
no = 0

# select one object at a time to see if it gets linked
for a in range(len(ssObjects)):
    testdf = obs_df.loc[objSplits[a],:]
    testdf = testdf.sort_values(by=['midPointTai'])
    # display(HTML(testdf.to_html()))

    # split by each unique mjd date of observation and put into nights
    splits = np.split(testdf['midPointTai'].values, np.where(np.abs(np.diff(testdf['midPointTai'].values)) >= 0.5)[0] + 1)
    splitra = np.split(testdf['ra'].values, np.where(np.abs(np.diff(testdf['midPointTai'].values)) >= 0.5)[0] + 1)
    splitdec = np.split(testdf['decl'].values, np.where(np.abs(np.diff(testdf['midPointTai'].values)) >= 0.5)[0] + 1)
    ras = [list(x) for x in splitra if len(x) > 1]
    decs = [list(x) for x in splitdec if len(x) > 1]
    nights = [list(x) for x in splits if len(x) > 1]
    obspernight = [len(ns) for ns in nights]



    # loop over each night and test for tracklets
    for n, night in enumerate(nights):
        if obspernight[n] < 2:
            pass
        else:
            for i in np.arange(len(night)):
                for j in np.arange(len(night)):
                    if j <= i:
                        pass
                    else:
                        diff = night[j] - night[i]
                        if diff > 0 and diff < 90/(60*24):
                            coord1 = SkyCoord([ras[n][i]], [decs[n][i]], unit=u.deg) 
                            coord2 = SkyCoord([ras[n][j]], [decs[n][j]], unit=u.deg)
                            sep = coord1.separation(coord2).arcsecond
                            if sep > 0.5:
                                tracklet = dict(
                                    no=no,
                                    ssObjectId=ssObjects[a],
                                    ra1=ras[n][i],
                                    dec1=decs[n][i],
                                    ra2=ras[n][j],
                                    dec2=decs[n][j],
                                    mjd1=night[i],
                                    mjd2=night[j]
                                )
                                tracklets.append(tracklet)
                                no += 1


linkdf = pd.DataFrame(tracklets)

nameLen = linkdf['ssObjectId'].str.len().max()
links = linkdf.to_records(
    index=False,
    column_dtypes=dict(no='f8', ssObjectId=f'a{nameLen}', ra1='f8', dec1='f8', ra2='f8', dec2='f8', mjd1='f8', mjd2='f8')
)

# split record array by object
i = np.argsort(links['ssObjectId'], kind='stable')
ssObjects, idx = np.unique(links['ssObjectId'][i], return_index=True)
objSplits = np.split(i, idx[1:])

# manually check which tracklet pairs are within 15 days of each other
for a in range(len(ssObjects)):
    testdf = linkdf.loc[objSplits[a],:]
    testdf = testdf.sort_values(by=['mjd1'])
    display(HTML(testdf.to_html()))
    linkage = 0
    for n, i in enumerate(np.arange(len(testdf))):
        for m, j in enumerate(np.arange(len(testdf))):
            if j <= i: # ensure we're not linking self with self or self with temporally past tracklets
                pass
            else:
                diff = testdf.iloc[j,:]['mjd2'] - testdf.iloc[i,:]['mjd1']
                if diff > 15: # if the difference is above 15 days, stop the checks and use mjds from obs that did meet criteria
                    idxs = np.arange(n, m)
                    if idxs.size != 0:
                        floors = [np.floor(x) for x in testdf.iloc[idxs,:]['mjd1']] # <-- makes sure that the tracklets are defined on seperate nights to avoid linking 3 tracklets on e.g. nights 1, 5.2, and 5.5
                        if len(np.unique(floors)) >= 3:
                            print('link between tracklet', testdf.iloc[i,:]['no'], testdf.iloc[i,:]['mjd1'], 'to tracklet', testdf.iloc[j-1,:]['no'], testdf.iloc[j-1,:]['mjd2'])
                    break

,no,ssObjectId,ra1,dec1,ra2,dec2,mjd1,mjd2
0,0,b'STC001TFa',9.703875,3.290640,9.703720,3.290564,60230.191531,60230.199118
1,1,b'STC001TFa',9.703875,3.290640,9.703236,3.290358,60230.191531,60230.222881
2,2,b'STC001TFa',9.703720,3.290564,9.703236,3.290358,60230.199118,60230.222881
3,3,b'STC001TFa',9.585765,3.241141,9.585273,3.240963,60236.070677,60236.094532
4,4,b'STC001TFa',9.585765,3.241141,9.584548,3.240653,60236.070677,60236.130676
5,5,b'STC001TFa',9.585273,3.240963,9.584548,3.240653,60236.094532,60236.130676
6,6,b'STC001TFa',9.526131,3.216260,9.525683,3.216062,60239.108595,60239.132505
7,7,b'STC001TFa',9.102824,3.042298,9.102653,3.042215,60264.016925,60264.029948
8,8,b'STC001TFa',9.102819,3.042303,9.102653,3.042215,60264.017372,60264.029948
9,9,b'STC001TFa',9.061745,3.025803,9.061331,3.025638,60267.086779,60267.110527


link between tracklet 0 60230.1915308476 to tracklet 6 60239.1325046928
link between tracklet 1 60230.1915308476 to tracklet 6 60239.1325046928
link between tracklet 2 60230.19911773316 to tracklet 6 60239.1325046928


,no,ssObjectId,ra1,dec1,ra2,dec2,mjd1,mjd2
11,11,b'STC001THa',222.067086,-23.157855,222.066626,-23.157745,60403.193904,60403.218698
12,12,b'STC001THa',222.064001,-23.157193,222.063495,-23.157089,60403.360875,60403.385730
13,13,b'STC001THa',221.912458,-23.122692,221.912286,-23.122641,60411.340329,60411.349004
14,14,b'STC001THa',221.872509,-23.112969,221.871994,-23.112850,60413.341137,60413.365709
15,15,b'STC001THa',221.872395,-23.112955,221.871994,-23.112850,60413.344248,60413.365709


,no,ssObjectId,ra1,dec1,ra2,dec2,mjd1,mjd2
16,16,b'STC001TJa',87.936286,22.517407,87.935761,22.517368,60322.075429,60322.099163
17,17,b'STC001TJa',87.936286,22.517407,87.935565,22.517358,60322.075429,60322.104528
18,18,b'STC001TJa',87.936286,22.517407,87.934999,22.517387,60322.075429,60322.128740
19,19,b'STC001TJa',87.935761,22.517368,87.935565,22.517358,60322.099163,60322.104528
20,20,b'STC001TJa',87.935761,22.517368,87.934999,22.517387,60322.099163,60322.128740
21,21,b'STC001TJa',87.935565,22.517358,87.934999,22.517387,60322.104528,60322.128740


,no,ssObjectId,ra1,dec1,ra2,dec2,mjd1,mjd2
22,22,b'STC001TKa',147.390625,14.53408,147.390186,14.534243,60316.280174,60316.302136


...

link between tracklet 8101 60235.15203993219 to tracklet 8107 60248.111364338714
link between tracklet 8102 60235.15203993219 to tracklet 8107 60248.111364338714
link between tracklet 8103 60235.15696125597 to tracklet 8107 60248.111364338714
link between tracklet 8104 60235.15696125597 to tracklet 8107 60248.111364338714
link between tracklet 8105 60236.15592251693 to tracklet 8109 60250.096260399056
link between tracklet 8106 60248.0872133825 to tracklet 8110 60260.02367140176
link between tracklet 8107 60248.09123868058 to tracklet 8110 60260.02367140176
link between tracklet 8108 60249.25389951513 to tracklet 8110 60260.02367140176
link between tracklet 8110 60260.01113901702 to tracklet 8112 60269.12356688455


,no,ssObjectId,ra1,dec1,ra2,dec2,mjd1,mjd2
8114,8114,b'STC0029va',134.980769,21.344153,134.980261,21.344314,60328.248836,60328.272638
8115,8115,b'STC0029va',134.958661,21.349836,134.958112,21.349978,60329.242560,60329.266905
8116,8116,b'STC0029va',134.756195,21.399998,134.755641,21.400141,60338.155748,60338.179475
8117,8117,b'STC0029va',134.709331,21.411239,134.708786,21.411358,60340.190238,60340.214069
8118,8118,b'STC0029va',134.456669,21.468919,134.456102,21.469030,60351.189248,60351.213028
8119,8119,b'STC0029va',134.456669,21.468919,134.456087,21.469044,60351.189248,60351.213473
8120,8120,b'STC0029va',134.456649,21.468918,134.456102,21.469030,60351.189693,60351.213028
8121,8121,b'STC0029va',134.456649,21.468918,134.456087,21.469044,60351.189693,60351.213473
8122,8122,b'STC0029va',134.050145,21.550256,134.049641,21.550322,60370.169116,60370.192851
8123,8123,b'STC0029va',133.538580,21.605660,133.538403,21.605635,60407.003341,60407.027179


link between tracklet 8114 60328.24883619729 to tracklet 8117 60340.21406938916
link between tracklet 8115 60329.24255965391 to tracklet 8117 60340.21406938916
link between tracklet 8116 60338.15574843053 to tracklet 8121 60351.213472500494


,no,ssObjectId,ra1,dec1,ra2,dec2,mjd1,mjd2
8126,8126,b'STC0029ya',320.123891,-16.332183,320.123646,-16.332250,60219.023969,60219.048461
8127,8127,b'STC0029ya',320.123891,-16.332183,320.123621,-16.332243,60219.023969,60219.048877
8128,8128,b'STC0029ya',320.123891,-16.332183,320.123350,-16.332324,60219.023969,60219.073416
8129,8129,b'STC0029ya',320.123646,-16.332250,320.123350,-16.332324,60219.048461,60219.073416
8130,8130,b'STC0029ya',320.123621,-16.332243,320.123350,-16.332324,60219.048877,60219.073416
8131,8131,b'STC0029ya',320.103539,-16.337380,320.103335,-16.337427,60221.029287,60221.051274
8132,8132,b'STC0029ya',320.084077,-16.342264,320.083837,-16.342320,60223.068536,60223.092898
8133,8133,b'STC0029ya',320.084077,-16.342264,320.083759,-16.342337,60223.068536,60223.101715
8134,8134,b'STC0029ya',320.084077,-16.342264,320.083527,-16.342394,60223.068536,60223.125844
8135,8135,b'STC0029ya',320.083837,-16.342320,320.083527,-16.342394,60223.092898,60223.125844


link between tracklet 8126 60219.02396924818 to tracklet 8137 60224.16674544148
link between tracklet 8127 60219.02396924818 to tracklet 8137 60224.16674544148
link between tracklet 8128 60219.02396924818 to tracklet 8137 60224.16674544148
link between tracklet 8129 60219.04846056165 to tracklet 8137 60224.16674544148
link between tracklet 8130 60219.04887722831 to tracklet 8137 60224.16674544148
link between tracklet 8131 60221.0292869633 to tracklet 8137 60224.16674544148
link between tracklet 8147 60518.33350572561 to tracklet 8162 60531.32753477625
link between tracklet 8148 60518.33350572561 to tracklet 8162 60531.32753477625
link between tracklet 8149 60518.35737537034 to tracklet 8162 60531.32753477625
link between tracklet 8150 60526.259763805894 to tracklet 8162 60531.32753477625
link between tracklet 8151 60526.259763805894 to tracklet 8162 60531.32753477625
link between tracklet 8152 60526.259763805894 to tracklet 8162 60531.32753477625
link between tracklet 8153 60526.26021